# Helper Functions

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import time
import datetime
import ta
import pickle
def upper_shadow(opener, close, high):
    if opener >= close:
        return abs(opener-high)
    else:
        return abs(close-high)

def lower_shadow(opener, close, low):
    if opener <= close:
        return abs(opener-low)
    else:
        return abs(close-low)

def real_body(opener, close):
    return abs(opener-close)

def total_dollar(av,bv,a,b):
    p = (a+b)/2.0
    v = av+bv
    return p*v

def log_dollar(a,b):
    p = (a+b)/2.0
    return np.log(p)

def volumen(av,bv):
    return av+bv

def price(a,b):
    g = (a+b)/2.0
    return b

prev = 1
def signer(x):
    global prev
    if x.values[-1] > x.values[0]:
        prev = 1
        return 1
    elif x.values[-1] < x.values[0]:
        prev = -1
        return -1
    else:
        return prev
#df = pd.read_csv('Dec.csv')
#df["Dollar"] = df.apply(lambda x: total_dollar(x.AskVolume,x.BidVolume,x.Ask,x.Bid), axis = 1)
def tick_preprocess(df):
    df["Price"] = df.apply(lambda x: price(x.Ask,x.Bid), axis = 1)
    return df
#df.to_csv("test_v6.csv",index=False)

In [ ]:
def tick_imbalance_candle(df):
    opener = []
    close = []
    high = []
    low = []
    tick_count = []
    tema = []

    start = 0
    cum_tsign = []
    csize = 1500
    for i, t in df.iterrows():
        cum_tsign += [t.tsign]
        jjj = t.tsign_ema
        if abs(sum(cum_tsign)) > abs(csize*jjj):
            buff_df = df.iloc[start:i+1]
            #print(str(start) + '----' + str(i+1))
            start = i + 1
            opener += [buff_df.Price.tolist()[0]]
            close += [buff_df.Price.tolist()[-1]]
            high += [max(buff_df.Price.tolist())]
            low += [min(buff_df.Price.tolist())]
            tema += [t.tsign_ema]
            tick_count += [len(buff_df.Ask.tolist())]
            cum_tsign = []
    start = 0
    imb = pd.DataFrame({'Open':opener,'High':high,'Low':low,'Close':close, 'tsign_ema':tema,'tick_count':tick_count})
    imb['us'] = imb.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
    imb['ls'] = imb.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
    imb['rb'] = imb.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
    return imb

In [ ]:
prev = 1
signs_cum = []
signs = []
q = 1
exp_size = 1500
boundary_labs = []
def signer(x):
    global prev
    if x.values[-1] > x.values[0]:
        prev = 1
        return 1
    elif x.values[-1] < x.values[0]:
        prev = -1
        return -1
    else:
        return prev

def bnd(x):
    global signs_cum
    global signs
    global q
    global exp_size
    global boundary_labs
    exp_size = 1500
    if abs(signs_cum[x.name]) > abs(exp_size * x.tsign_ema):
        if x.name+1 < len(signs_cum):
            q = q+1
            signs_cum[x.name+1:] = list(np.cumsum(signs[x.name+1:]))
            boundary_labs[x.name+1:] = [q] * len(boundary_labs[x.name+1:])
        return 'break' + '--' + str(abs(signs_cum[x.name])) + '--' + str(abs(exp_size * x.tsign_ema))
    else:
        return 'pass'
    
def tick_imbalance_candle_v2(df):
    global signs_cum
    global signs
    global q
    global exp_size
    global boundary_labs
    df['tsign_cum'] = df.tsign.cumsum()
    signs_cum = df.tsign_cum.tolist()
    signs = df.tsign.tolist()
    boundary_labs = [q] * len(df)
    df.reset_index(drop = True, inplace = True)
    df['boundary'] = df.apply(lambda x: bnd(x), axis = 1)
    df['tsign_cum_v2'] = signs_cum
    df['boundary_labs'] = boundary_labs
    signs_cum = []
    sizes = []
    df['shifted_boundary'] = df.boundary_labs.shift(1)
    df['grouper_check'] = df.shifted_boundary != df.boundary_labs
    df['grouper'] = df.grouper_check.cumsum()
    fin_df = df.groupby('grouper')['Ask'].agg(['min', 'max', 'first', 'last', 'size'])
    fin_df.columns = ['Low', 'High', 'Open', 'Close', 'Size']
    fin_df['us'] = fin_df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
    fin_df['ls'] = fin_df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
    fin_df['rb'] = fin_df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
    df = pd.DataFrame({'a':[]})
    return fin_df
    
    
    

In [ ]:
def candle_form(df):
    opener = []
    high = []
    low = []
    close = []
    rb = []
    us = []
    ls = []
    dur = []
    counter = 0
    price = df.Price.tolist()

    buff = 0
    start = 0
    min_max = df.groupby(np.arange(len(df))//300)['Price'].agg(['min','max'])
    op = df.groupby(np.arange(len(df))//300)['Price'].first()
    cl = df.groupby(np.arange(len(df))//300)['Price'].last()
    df = pd.concat([min_max, op,cl], axis = 1)
    df.columns = ['Low','High','Open','Close']
    df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
    df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
    df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)

    return df

In [ ]:
def indicator_fill(df):
    a = ta.trend.ema_indicator(close=df.Close,window=5)
    f = ta.trend.sma_indicator(close=df.Close,window=50)
    i = ta.trend.sma_indicator(close=df.Close,window=20)
    c = ta.volatility.average_true_range(high=df.High,low=df.Low,close=df.Close)
    df["5_EMA"] = a
    df["50_SMA"] = f
    df["20_SMA"] = i
    return df


In [ ]:
from sklearn.preprocessing import LabelEncoder

side_le = LabelEncoder()



def label_calc_v2(df):
    df['last_close'] = df.Close.rolling(window = 10).apply(lambda x: x.values[-1])
    df.last_close = df.last_close.shift(-10)
    df = df[:-10]
    df['Labels'] = df.apply(lambda x: 'Buy' if x.last_close > x.Close else 'Sell', axis = 1)
    df['Labels'] = side_le.fit_transform(df['Labels'].tolist())
    with open("side_label_le_tick_spx.pkl","wb") as f:
        pickle.dump(side_le,f)
    return df
    

## Vectorizing function that prepares the training data and labels

In [ ]:
vec = []
labs = []
from numpy_ext import rolling_apply
def veccr(c,h,l,o,rb,ls,us,ma):
    global vec
    vec += [np.hstack((c,h,l,o,rb,ls,us,ma))]
    return 0

def labr(x):
    global labs
    labs += [x.values[-1]]
    return 0

def df_vectorize_v2(df):
    side = []

    
    global vec
    global labs
    
    rolling_apply(veccr, 10, df.Close.values, df.High.values, df.Low.values, df.Open.values, df.rb.values, df.ls.values
                 , df.us.values, df['20_SMA'].values)
    
    
    df.Labels.rolling(10).apply(labr)
    side = labs
    labs = []
    
    fin_vec = np.array(vec)
    vec = []
    
    return fin_vec, side

# Crawling the tick data and converting them in to training data using the above helper functions

In [ ]:
import os
df_collect = []
vectors = []
sides = []
metas = []
bmv = []
smv = []
bml = []
sml = []
from fracdiff import FracdiffStat
cf = FracdiffStat()
hf = FracdiffStat()
of = FracdiffStat()
lf = FracdiffStat()
mf = FracdiffStat()
import warnings
warnings.filterwarnings('ignore')
for root, dirs, files in os.walk(r'C:\Users\91984\Desktop\Bot ML\Training Data\Ticks Data S&P500 - Copy'):
    if not files:
        continue
    else:
        for f in files:
            df_collect += [pd.read_csv(root+'\\'+f)]
        df = pd.concat(df_collect)
        df.reset_index(drop = True, inplace = True)
        df = tick_preprocess(df)
        df = candle_form(df)
        if(len(df)<50):
            continue
        df = indicator_fill(df)
        df = df[20:]
        df = label_calc_v2(df)
        df.Close = list(np.squeeze(cf.fit_transform(np.array(df.Close).reshape(-1,1)), axis = 1))
        df.High = list(np.squeeze(hf.fit_transform(np.array(df.High).reshape(-1,1)), axis = 1))
        df.Open = list(np.squeeze(of.fit_transform(np.array(df.Open).reshape(-1,1)), axis = 1))
        df.Low = list(np.squeeze(lf.fit_transform(np.array(df.Low).reshape(-1,1)), axis = 1))
        df['20_SMA'] = list(np.squeeze(mf.fit_transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
        df = df[1:]
        #df = frac_diff_calc(df)
        v, s = df_vectorize_v2(df)
        vectors += [v]
        sides += s
        df_collect = []
        print(f + ' ' + str(vectors[0].shape))

## Finalizing and storing the training data

In [ ]:
train_vec = np.vstack((vectors))
side_lab = sides
print(train_vec.shape)



In [ ]:
from collections import Counter

side_lab = list(map(lambda x: int(x), side_lab))

ctr2 = Counter(side_lab)

print(ctr2)


# SMOTE Class Imbalance Adjustment

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NearMiss
ov1 = SMOTE()
undersample = NearMiss(version=3, n_neighbors_ver3=3)
#train_vec = np.hstack((train_vec,side_lab.reshape(-1,1)))
train_vec = np.nan_to_num(train_vec)
side_vec, side_lab = undersample.fit_resample(train_vec,side_lab)
#mv, ml = ov2.fit_resample(train_vec,meta_lab)
#smv, sml = ov3.fit_resample(smv,sml)

In [ ]:
ctr2 = Counter(side_lab)

print(ctr2)

## Loading and processing the training data

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
#fin_vec, labels = sklearn.datasets.load_svmlight_file("fracdiff_side_prediction.txt.train")

side_lab = np.array(side_lab, dtype='int32')
print(side_lab.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_vec, side_lab, test_size=0.1, random_state=42, stratify=side_lab)
print(X_train.shape)
print(y_train.shape)


# XGBOOST SKLEARN API training

In [ ]:
from xgboost import XGBClassifier
from joblib import dump
from joblib import load
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 1000, max_depth = 100, max_features = 0.1, bootstrap = True, 
                            n_jobs = -1, verbose = 2, max_samples = 0.05, random_state = 42)
xgbclasser = XGBClassifier(max_depth=100, objective='multi:softprob', use_label_encoder=False,num_class=2,n_estimators=100)
#xgbclasser.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=30)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)


In [ ]:
from joblib import dump
dump(rf, 'RF_SPX_tick_classifier.joblib')

In [ ]:
import math
X_test = np.nan_to_num(X_test)
rf.score(X_test,y_test)

In [ ]:
bc = BaggingClassifier(n_estimators = 200, max_features = 0.25, bootstrap = True, 
                            n_jobs = -1, verbose = 2, max_samples = 0.2, random_state = 42, bootstrap_features= True)
bc.fit(X_train,y_train)
bc.score(X_test,y_test)

In [ ]:
#dump(xgbclasser, "fracdiff_side_prediction_xgbclasser.model")
dump(xgbclasser, "10_lookback_1500_tick_imbalance_spx.model")

# 50 lookback predictions

In [ ]:
import os
df_collect = []
vectors = []
sides = []
metas = []
bmv = []
smv = []
bml = []
sml = []
for root, dirs, files in os.walk(r'C:\Users\91984\Desktop\Bot ML\FX_Advanced_ML\Jan 2021 S_P 500'):
    if not files:
        continue
    else:
        for f in files:
            df_collect += [pd.read_csv(root+'\\'+f)]
        df = pd.concat(df_collect)
        df.reset_index(drop = True, inplace = True)
        df = tick_preprocess(df)
        df = candle_form(df)
        if(len(df)<50):
            continue
        df = indicator_fill(df)
        df = df[20:]
        df = label_calc_v2(df)
        df.Close = list(np.squeeze(cf.fit_transform(np.array(df.Close).reshape(-1,1)), axis = 1))
        df.High = list(np.squeeze(hf.fit_transform(np.array(df.High).reshape(-1,1)), axis = 1))
        df.Open = list(np.squeeze(of.fit_transform(np.array(df.Open).reshape(-1,1)), axis = 1))
        df.Low = list(np.squeeze(lf.fit_transform(np.array(df.Low).reshape(-1,1)), axis = 1))
        df['20_SMA'] = list(np.squeeze(mf.fit_transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
        df = df[1:]
        #df = frac_diff_calc(df)
        v, s = df_vectorize_v2(df)
        vectors += [v]
        sides += s
        df_collect = []
        print(f + ' ' + str(vectors[0].shape))

In [ ]:
from joblib import load
import pickle
model_1 = load("10_lookback_1500_tick_imbalance_spx.model")


with open("side_label_le_tick_spx.pkl","rb") as f:
    side_le = pickle.load(f)



test_s = np.array(sides)
test_v = np.vstack((vectors))
test_v = np.nan_to_num(test_v)
preds = rf.predict(test_v)
probs = np.max(rf.predict_proba(test_v),axis=1)

probs = list(probs)
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print(accuracy_score(test_s, preds))
print(classification_report(test_s, preds))

predictions = preds.astype(int)
predictions = side_le.inverse_transform(list(predictions))






ab_df = pd.DataFrame({"Orig":sides,"Pred":predictions,"Pred_Probs":probs})
ab_df.to_csv("rf_tick_spx_output.csv",index=False)

In [ ]:
print(len(vectors))

In [ ]:
#bst = xgb.Booster({'nthread':4})
#bst.load_model('side_prediction_xgb.model')
#side_predict_sk = load('fracdiff_side_prediction_xgbclasser.model')
side_predict_sk = load('fracdiff_side_prediction_xgbclasser_interaction_features.model')
#bm_predict_sk = load('buy_meta_prediction_xgbclasser.model')
#sm_predict_sk = load('sell_meta_prediction_xgbclasser.model')


pred_side_sk = side_predict_sk.predict(tv)
prob_side_sk = np.max(side_predict_sk.predict_proba(tv), axis = 1)

#buy_pred_meta_sk = bm_predict_sk.predict(bmvp)
#buy_prob_meta_sk = np.max(bm_predict_sk.predict_proba(bmvp), axis = 1)

#sell_pred_meta_sk = sm_predict_sk.predict(smvp)
#sell_prob_meta_sk = np.max(sm_predict_sk.predict_proba(smvp), axis = 1)

In [ ]:
side_predict_sk = load('fracdiff_side_prediction_xgbclasser_interaction_features.model')
df = pd.read_csv('Jan_Test.csv')
df = tick_preprocess(df)
df = candle_bar(df)
df = indicator_fill(df)
df, _, _ = label_calc(df)
df = frac_diff_calc(df)

test_v, test_s, test_m = df_vectorize(df)
test_v = np.array(test_v)
preds = side_predict_sk.predict(test_v)
probs = np.max(side_predict_sk.predict_proba(test_v), axis = 1)

preds = preds.astype(int)
preds = side_le.inverse_transform(list(preds))
probs = list(probs)
df1 = df[20:]
df1["SK_Side_Prediction"] = preds
df1["SK_Side_Probability"] = probs
df1.to_csv('Jan_Test_preds.csv', index = False)

In [ ]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print(accuracy_score(ts, pred_side_sk))
print(classification_report(ts, pred_side_sk))

# Stacking Classifier

In [ ]:
from xgboost import XGBClassifier
from joblib import dump
from joblib import load
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
xgclass = XGBClassifier(max_depth=100, objective='multi:softprob', use_label_encoder=False,num_class=4, n_estimators = 10)
rf = RandomForestClassifier(n_estimators=10, random_state=42)
lr = LogisticRegression()
lvc = LinearSVC(random_state=42)
bc = BaggingClassifier(random_state=42)
#estimators = [('rf', rf),('xgb', xgclass),('lr', lr)]
estimators = [('rf', rf),('lvc', lvc),('bc', bc),('lr', lr)]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

clf.fit(side_vec,side_lab)

In [ ]:
dump(clf,'StackingClassifier.DAT')

In [ ]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print(accuracy_score(ts, pred_side_sk))
print(classification_report(ts, pred_side_sk))

In [ ]:

print(len(df))
df1 = df[50:]
print(len(df1))
print(len(df))
#bdf = buy_df[50:]
#sdf = sell_df[50:]

In [ ]:


pred_side_sk = pred_side_sk.astype(int)
pred_side_sk = side_le.inverse_transform(list(pred_side_sk))
prob_side_sk = list(prob_side_sk)

#buy_pred_meta_sk = buy_pred_meta_sk.astype(int)
#buy_pred_meta_sk = meta_le.inverse_transform(list(buy_pred_meta_sk))
#buy_prob_meta_sk = list(buy_prob_meta_sk)

#sell_pred_meta_sk = sell_pred_meta_sk.astype(int)
#sell_pred_meta_sk = meta_le.inverse_transform(list(sell_pred_meta_sk))
#sell_prob_meta_sk = list(sell_prob_meta_sk)


#df1["BST_Side_Prediction"] = pred_bst
df1["SK_Side_Prediction"] = pred_side_sk
df1["SK_Side_Probability"] = prob_side_sk
#bdf["SK_Meta_Prediction"] = buy_pred_meta_sk
#bdf["SK_Meta_Probability"] = buy_prob_meta_sk
#sdf["SK_Meta_Prediction"] = sell_pred_meta_sk
#sdf["SK_Meta_Probability"] = sell_prob_meta_sk
df1.to_csv('Jan_Test_preds.csv', index = False)
#bdf.to_csv('Jan_Test_buy_meta.csv', index = False)
#sdf.to_csv('Jan_Test_sell_meta.csv', index = False)

## positional encoding for transformer encoder

In [ ]:
pe = []
inner = []
import math
import numpy as np
for p in range(550):
    for i in range(0,8,2):
        inner += [math.sin(p/(10000**(i/32)))]
        inner += [math.cos(p/(10000**(i/32)))]
    
    pe += [inner]
    inner = []
pe = np.array(pe)
pe = np.expand_dims(pe,axis=0)
pe = np.repeat(pe,fin_vec.shape[0],axis = 0)
print(pe.shape)
#xt1 = np.expand_dims(X_train,axis = 2)


In [ ]:
import sklearn
from sklearn import datasets
fin_vec, labels = sklearn.datasets.load_svmlight_file("side_prediction.txt.train")
print(fin_vec.shape)
#X_train_positional = np.add(pe,X_train)
#print(X_train_positional.shape)

In [ ]:
fin_vec = fin_vec.toarray()
print(fin_vec.shape)

In [ ]:
fin_vec = np.repeat(fin_vec,8, axis = 2)
print(fin_vec.shape)

In [ ]:
fin_vec = fin_vec + pe

In [ ]:
import torch
import torch.nn as nn

encoder_layer = nn.TransformerEncoderLayer(d_model=8, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
src = torch.rand(550, 32, 8)
out = transformer_encoder(src)
print(out.shape)
fwd = nn.Linear(550*8,128)
out = fwd(out.view(out.shape[1],-1))
print(out.shape)

# Transformer Encoder

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer
import numpy as np
from torchvision import transforms,models,datasets
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from torchvision import transforms,models,datasets
class Tick_encoder(nn.Module):
    def __init__(self):
        super(Tick_encoder, self).__init__()

        
        self.encoder = nn.TransformerEncoderLayer(d_model=8, nhead=8)
        self.trans_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc1 = nn.Linear(550 * 8, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)


    def forward(self, state):
        
        out = self.trans_encoder(state)
        out = F.relu(self.fc1(out.view(out.shape[1],-1)))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)

        return out

In [ ]:
model = Tick_encoder()

In [ ]:

X_t = T.FloatTensor(fin_vec)
labs = T.LongTensor(labels)
print(X_t.shape)
train_data = TensorDataset(X_t,labs)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
save_location = 'C:\\Users\\91984\\Desktop\\Bot ML\\FX_Advanced_ML\\Torch_Encoder_Model'

#model = T.load(save_location+'\\'+'GAF_CNN_trading_model_target.pth')
trainer = optimizer.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
for e in range(20):
    running_loss = 0
    acc = 0
    for i, (batch,labs) in enumerate(train_loader):
        
        trainer.zero_grad()
        out1 = model(batch.reshape(550,-1,8))
        loss = criterion(out1,labs)
        loss.backward()
        trainer.step()
        running_loss += loss.item()
        out1 = T.nn.functional.softmax(out1,dim=1)
        Conf_score,prediction = T.max(out1,1)
        tot_score = sum(x.item()==y.item() for x,y in zip(prediction,labs))
        acc += tot_score/len(labs)
        if i%1==0:
            print('Processing batch '+str(i+1)+' of '+str(98273//32)+
                  ' batches in Epoch '+str(e+1)+' with Loss: '+
                  str(running_loss/200)+' with accuracy: '+str(acc/200))
            running_loss = 0
            acc = 0
print('Saving The Model')
T.save(model,save_location+'\\'+'Tick_transformer_Encoder.pth')
print('Training Done')

In [ ]:
plt.plot(df1.pos_run_agg_EMA_5.tolist()[200:240], label="line1")
plt.plot(df1.pos_run_agg_SMA_100.tolist()[200:240], label="line2")
plt.legend()
plt.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

buy_go = pd.read_csv("buy_go.csv")
buy_no_go = pd.read_csv("buy_no_go.csv")
sell_go = pd.read_csv("Sell_Go.csv")
sell_no_go = pd.read_csv("Sell_No_Go.csv")
buy_near = pd.read_csv("Buy_near_sell.csv")



In [ ]:
fig = go.Figure(data=[go.Candlestick(open=buy_go['Open'], high=buy_go['High'],
                low=buy_go['Low'], close=buy_go['Close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
pe